In [1]:
import os
import traceback
from collections import Counter
import numpy as np
import pandas
import utils

In [2]:
INPUT_ROOT='/home/dhananjay/hopsworks_project/KI/raw_sample/raw'
OUTPUT_ROOT='/home/dhananjay/hopsworks_project/KI/raw_sample'
REPORT_FILE='report_diamond_raw_stats.txt'
FILTERED_PREFIX='_filtered'

In [3]:
all_files=[]
for path, folders, files in os.walk(INPUT_ROOT):
    for file in files:
        all_files.append(os.path.join(path, file))

print('input files ', len(all_files))

input files  9759


In [4]:


THREASHOLD_PERCENT=90


    
"""
Table to create statistics. Columns of report file are :

FILE_NAME | SAMPLE_NAME | TOTAL_READS | UNIQUE_READS | UNIQUE_PAIR_READS | HPV Count Exclusive | HPV-TYPES (non overlapping uniques)
| HPV-TYPES (HPV Coverage-Pair) | Coverage per SubjectType | Positive

Returns:
List with each item corresponding to 1 row for report
"""
def table_collect_stats(file_path):
    
    file_name=os.path.basename(file_path)
    
    print('processing file: ',  file_path)
    try : 
        if FILTERED_PREFIX in file_path:
            df=pandas.read_csv(file_path, sep='\t', names=["Query", "Subject", "Percentage", "Alignment",'Mismatch','Gap','Start','End','SA','EA','EV','BitScore'],header=1)
        else :
            df=pandas.read_csv(file_path, sep='\t', names=["Query", "Subject", "Percentage", "Alignment",'Mismatch','Gap','Start','End','SA','EA','EV','BitScore'],header=None)
        
        totalRows=df.shape[0]      
        queries=df.Query
        subjects=df.Subject      
        unique_queries=queries.unique()
        # unique reads
        unique_count=len(unique_queries)
        # subject types           
        subjects_filtered = get_subject_types(subjects)                              
        df['SubjectType']=subjects_filtered      
        # protein type 
        protein_filtered=[ x.split('|')[1].split('.')[1] for x in subjects ] 
        df['ProteinType']=protein_filtered 
        # unique subject types
        groupedQuerySubject=df[['Query','SubjectType']].groupby('SubjectType',sort=False).count().sort_values(by=['Query'],ascending=False)
        # protein type-subject type grouped
        groupedProteinSubject=df[['Query','SubjectType','ProteinType']].groupby(['SubjectType','ProteinType']).count().sort_values(by=['Query'],ascending=False)
           
        subjects_conc=''
        subjects_unique_conc=''
        subjects_cov_len=''
        unique_queries_set=set(unique_queries)
        uniqueQueries_set_reducing=set(unique_queries.copy())
        subject_unique_reads={} #  {subjectType: uniqueReadsCount}
        coverage_per_sub_type={} # {subjectType: totalCoverageLen}
        is_positive=False
        subject_total_cov_len=''

        #  HPV counts and HPV non overlapping counts
        for index, row in groupedQuerySubject.iterrows():            
            current_reads=df[df.SubjectType == index].Query
            # for unique HPV counts
            unique_reads_count=exclusive_reads_count(df,index)
            
            subject_unique_reads[index] = unique_reads_count # add to dict 
            subjects_conc = subjects_conc+index+'='+str(unique_reads_count)+';' # HPV exclusive count
            # for unique non overlapping HPV counts
            unique_currentReads=uniqueQueries_set_reducing.intersection(current_reads)          
            uniqueQueries_set_reducing=uniqueQueries_set_reducing.difference(unique_currentReads)                
            subjects_unique_conc = subjects_unique_conc+index+'='+str(len(unique_currentReads))+';'           

      
        # Coverage length
        for index,row in groupedProteinSubject.iterrows():   
            subjectType = index[0]
            protein =  index[1]
            alignmentDf = df[(df.SubjectType ==  subjectType ) & (df.ProteinType == protein )]
            alignmentPositions=alignmentDf[['SA','EA']].values.tolist()            
            coverageLen_current=count_merged_intervals(alignmentPositions)                   
            subjects_cov_len=subjects_cov_len+index[0]+'|'+index[1]+'='+str(coverageLen_current)+';' 
            covLenPerSub=coverage_per_sub_type.get(subjectType)
            if covLenPerSub:
                coverage_per_sub_type[subjectType]=covLenPerSub+coverageLen_current
            else :
                coverage_per_sub_type[subjectType]=coverageLen_current
                        
       
         # total cov len per subject type
        for subType,covLen in coverage_per_sub_type.items():
            subject_total_cov_len = subject_total_cov_len+subType+'='+str(covLen)+';'  
            # check for positive
            reads=subject_unique_reads.get(subType)
            if not is_positive:
                is_positive=check_positive(reads,covLen)                        


        # read pairs count
        querySubject=df[['Query','SubjectType']].sort_values(by=['Query'])
        queries=querySubject.Query
        subjects=querySubject.SubjectType
        r1=[]
        r2=[]
        for count,item in enumerate(queries):
            seq=item.split('/')[0]
            if '/1' in item:            
                r1.append((seq,subjects[count]))
            elif '/2':
                r2.append((seq,subjects[count]))

        r1_dict = dict(Counter(r1))
        r2_dict = dict(Counter(r2))
        pairs=0 # number of unique pairs
        for r1_item in r1_dict.items():      
            r1_key=r1_item[0]
            r1_value=r1_item[1]  
            if r1_key in r2_dict:               
                r2_value=r2_dict.get(r1_key)                      
                pairs+=min([r1_value,r2_value])
                
    
                
        # get original sample name
        sample=os.path.splitext(file_name)[0].replace('_NH_unmapped','').replace('Hpv_trim_','').replace('_filtered','')

        # concatenate values into string
        line=file_name+'\t'+sample+'\t'+str(totalRows)+'\t'+str(unique_count)+'\t'+str(pairs)+'\t'+subjects_conc+'\t'+subjects_unique_conc+'\t'+subjects_cov_len \
        +'\t'+str(subject_total_cov_len)+'\t'+str(is_positive)

        return line
    except Exception as e: 
        print(e)
        traceback.print_exc()
        return None
    
def exclusive_reads_count(df,subject_type):
    current_reads=set(df[df.SubjectType == subject_type].Query.unique())
    other_types_reads=set(df[df.SubjectType != subject_type].Query.unique())
    unique_reads_count=len(current_reads.difference(other_types_reads))
    return unique_reads_count
        

def count_merged_intervals(intervals, sort=True):
    '''Gets a list of intervals (should be sorted or marked to be sorted with sort=True), 
       merges them when overlapping and then calculate the sum (including limits).
       Original list: [4, 8], [10, 12], [1, 5]
       Sorted and merged: [1, 8], [10, 12]
       Returns: 11
    '''
    intervals = np.array(intervals)
    if sort:
        intervals.sort(axis=0)
    starts = intervals[:, 0]
    ends = np.maximum.accumulate(intervals[:, 1])
    valid = np.zeros(len(intervals) + 1, dtype=np.bool)
    valid[0] = True
    valid[-1] = True
    valid[1:-1] = starts[1:] >= ends[:-1]
    merged = np.vstack((starts[:][valid[:-1]], ends[:][valid[1:]])).T
    # resultado2 = np.zeros(len(merged), dtype=np.int)
    resultado2 = merged[:, 1] - merged[:, 0] + 1
    return np.sum(resultado2)


def check_positive(reads,coverage,threshold_unique_count=10,threshold_coverage=230):
    
    if reads > threshold_unique_count and coverage > threshold_coverage:
        return True
    return False


def get_subject_types(subjects):
    """
    Split Subjects column into corresponding subject types
    :param subjects: List
    :return: subjects_filtered:List
    """
    splt_char='-'
    K=2 # index for splitted array
    subjects_filtered=[]
    for item in subjects:
        sub_grp=item.split('|')[-2]
        splits_arr = sub_grp.split(splt_char)
        if len(splits_arr)==3: # incase of exceptions e.g. HPV-mSK152
            sub = splt_char.join(splits_arr[:K])
        else :
            sub = splits_arr[0]
        subjects_filtered.append(sub)
    
    return subjects_filtered


#table_collect_stats('/home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_004660a1-aaed-4619-87a0-e8260f09128a_gdc_realn_rehead_NH_unmapped')


In [5]:
#table_collect_stats('/home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_008d4eb5-91d6-4c69-b24b-afe15ff18c17_gdc_realn_rehead_NH_unmapped')
#table2('hdfs:///Projects/TCGA_viruses/Diamond/raw/Hpv_trim_004660a1-aaed-4619-87a0-e8260f09128a_gdc_realn_rehead_NH_unmapped')
#table2('hdfs:///Projects/TCGA_viruses/Diamond/raw/Hpv_trim_09f31b55-df37-4673-8a8c-bde9bc8780c4_gdc_realn_rehead_NH_unmapped')



In [6]:


# list of local file paths

In [7]:
outputLines=list(map(lambda x:table_collect_stats(x), all_files))
len(outputLines)

processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_34994ab9-7ace-4a11-a4bf-201e30797d9c_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_47d54a94-5ef2-4507-ad22-3b3ed85f71fd_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_3c920de9-10c2-4429-9cda-a4146fe5f677_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_959df651-e814-4dcf-8833-0827a2a6219f_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_04403717-4cbe-4443-b114-96901370eeef_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_817a0c09-a8cd-436e-8674-257d9f3a4e05_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_7ad89704-da2a-43b8-9cc9-774822fe107e_gdc_realn_rehe

processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_a70333fe-8fa4-4403-9ba0-c2fa2bcd7b58_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_8a08a406-452e-4c8d-adc5-e95e1f00c042_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_a3e8a207-485f-4edd-8aeb-23363f460073_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_21288968-5618-451c-9df6-0e8366d12f1e_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_f6ad3404-02e4-43cf-9635-53194a0ce6eb_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_b71af913-2d1c-4f56-91f7-ab2b267dc2d0_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_a0df38b2-2107-4766-a356-85ae40a66d71_gdc_realn_rehe

processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_1e5cdc86-6546-47fd-91dd-db011bec3267_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_56089d6d-8b99-4325-8e68-e91d78b56322_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_aa44f578-7400-4ef8-b1b0-8c369bfda870_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_640428c8-5f07-48ef-bab1-6be3fd24a59b_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_897a204b-0157-4587-88eb-5e7fdd18a3a1_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_b0d777ee-bb4a-4906-a046-8a69b0752003_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_a004383e-3526-4894-bd4d-0d2b74210b67_gdc_realn_rehe

processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_0b97c257-b259-43ae-9056-173318e140a5_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_e29dade9-6c35-48d3-b203-e1413abca8ec_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_510972a1-f3a9-46e1-b7d5-354bd0de7944_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_232df2d3-b8af-4947-8ca0-70225cc9179c_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_d6404a23-9ad8-4bf2-9744-24588e2e5ddb_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_08efe2e7-edc4-4c77-878f-558f3b473e87_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_c87020b9-b531-4fa4-9b5c-5e06734ff734_gdc_realn_rehe

processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_0544be4b-3434-4a1b-a280-3d05888669f5_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_3ed8e8f8-34bf-4c54-8032-e79a1ac87af7_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_b27384f2-6862-4de3-8f29-009e2355ab1e_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_4b6f1599-6c85-438e-a08b-cdf04449b9fb_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_fae1853b-d9b9-473d-8a0b-5ef408cc08c1_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_cea4b42a-8d48-43f2-96a3-53f32b0f0c5e_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_a8c3b6b6-f12d-48e1-a3ea-5a5ff716c403_gdc_realn_rehe

...

processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_b7ad37f8-3647-4439-be16-3edeb1dd312e_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_1c18bed9-7c00-4230-9297-c38b339558df_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_566d299b-0a83-4e72-adab-d3f9e02312b8_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_c61593b0-280b-414e-9831-39928a6817b0_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_79fb9497-3945-4e1d-bde0-a9b8f072b213_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_227585f0-c4a0-4aab-a545-64ef7354ae99_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_4786976b-f84e-4975-90fa-9cce9b86e691_gdc_realn_rehe

processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_6b4a5b42-abe0-478e-bf39-408ce9451aee_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_f4daa5b9-f2a5-44c1-9d18-da1577e343f5_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_57ca2851-3cb8-413a-a1f9-23c0b84fde9b_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_900bd382-b71b-4424-8609-46b18f2f31b2_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_296c9663-e022-4840-94e1-378c7d486f51_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_539addd4-4b59-499f-8918-5c281f28f2c2_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_4a4d144a-bdd7-4ca5-8cb9-7b6eca2750a4_gdc_realn_rehe

processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_5b9a2d0f-22f8-40d7-92ba-112e4b4c3906_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_43cee8b5-c602-4190-9c7d-56ddfd2c490a_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_07e3020b-b155-4d5a-b13f-96c00ae8d8d9_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_0eb189b3-3ad2-4875-b1ae-df2117455faf_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_6519c9c4-daf3-47cc-9091-4d48dbe50aee_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_f28e56d2-f0ab-4b09-8a96-790eee7b9188_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_8caa5470-0f85-4cad-a8fa-59622ef06ac1_gdc_realn_rehe

processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_39a21dbf-0712-4b4c-ad55-6e2d779502d5_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_902c18b9-dad7-4e45-a969-d4a8ac916915_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_284b035e-c687-49f8-beec-e64c28297a71_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_d54c7372-5a68-426e-901a-5c63a2ac0dbd_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_0a4fbb80-d2b3-4e72-942a-dd2e2964ec0e_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_7601de2e-8f65-45c8-9e16-7cec08b9eba0_gdc_realn_rehead_NH_unmapped
processing file:  /home/dhananjay/hopsworks_project/KI/raw_sample/raw/Hpv_trim_6bccd083-a96d-4ca1-a3cf-84bf111df514_gdc_realn_rehe

9759

#### Write into file

In [8]:
with open(REPORT_FILE, 'w') as f:
    f.write('File Name'+'\t'+'Sample Name'+'\t'+'Total Reads'+'\t'+'Unique Reads'+'\t'+'Unique pairs'+'\t'+'HPV Count Exclusive'+'\t'+'HPV types(non-overlapping uniques)'
            +'\t'+'HPV Coverage-Pair'+'\t'+'Coverage per SubjectType'+'\t'+'Positive')
    f.write('\n')
    for i in outputLines:
        f.write(i+'\n')

        
